In [2]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np

from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.layers.merge import add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout

# small library for seeing the progress of loops.
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

C:\Users\DELL\AppData\Local\Temp\ipykernel_11748\3791287272.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive
def load_doc(filename):
    # Opening the file as read only
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text


Mounted at /gdrive
/gdrive


In [ ]:
# get all imgs with their captions
def all_img_captions(filename):
    file = load_doc(filename)
    captions = file.split('\n')
    descriptions ={}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [ caption ]
        else:
            descriptions[img[:-2]].append(caption)
# to show  the key of the single image
    # print(caption)
    # res = list(descriptions.keys())[0]
    # print(str(res))
    return descriptions

In [ ]:
#Data cleaning- lower casing, removing puntuations and words containing numbers
def cleaning_text(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):
            img_caption.replace("-"," ")
            desc = img_caption.split()
            #converts to lowercase
            desc = [word.lower() for word in desc]
            #remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            #remove hanging 's and a 
            desc = [word for word in desc if(len(word)>1)]
            #remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            #convert back to string
            img_caption = ' '.join(desc)
            captions[img][i]= img_caption
    return captions

In [ ]:
def text_vocabulary(descriptions):
    # build vocabulary of all unique words
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab

In [ ]:
#All descriptions in one file 
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc )
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()

In [ ]:

root_captioning = "/gdrive/My Drive/image caption generator/text"
root_caption = "/gdrive/My Drive/image caption generator"
root_img = "/gdrive/My Drive/image caption generator/Images"


In [ ]:
#we prepare our text data
filename = os.path.join(root_captioning,\
            'Flickr8k.token.txt')
#loading the file that contains all data\
#mapping them into descriptions dictionary img to 5 captions
descriptions = all_img_captions(filename)
# use to show the elements in each key ie caption of each image
# print(descriptions)
# for key in descriptions:
#     print(key, ' : ', *descriptions[key],sep='\n')
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)
# for key in clean_descriptions:
#     print(key, ' : ', *clean_descriptions[key],sep='\n')
#building vocabulary 
vocabulary = text_vocabulary(clean_descriptions)
# print(*vocabulary,sep='\n')
print("Length of vocabulary = ", len(vocabulary))
#saving each description to file 
save_descriptions(clean_descriptions,os.path.join(root_caption,\
                                                  'descriptions.txt'))


Length of descriptions = 8092
Length of vocabulary =  8763


In [ ]:
def extract_features(directory):
        model = Xception( include_top=False, pooling='avg' )
        features = {}
        for img in tqdm(os.listdir(directory)):
            filename = directory + "/" + img
            image = Image.open(filename)
            image = image.resize((299,299))
            image = np.expand_dims(image, axis=0)
            # image = preprocess_input(image)
            image = image/127.5
            image = image - 1.0

            feature = model.predict(image)
            features[img] = feature
        return features

#2048 feature vector
features = extract_features(os.path.join(root_img))
dump(features, open("Featurespro.p","wb"))

In [ ]:
# loading the feature file  since we have the feature.p file 
features = load(open(os.path.join(root_caption,\
                                  'features.p'),"rb"))

In [ ]:
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos

In [ ]:
def load_clean_descriptions(filename, photos): 
    #loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words)<1 :
            continue
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + ' <end>'
            descriptions[image].append(desc)
        return descriptions

In [ ]:
def load_features(photos):
    #loading all features
    all_features = load(open(os.path.join(root_caption,\
                                  'features.p'),"rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    # view the feature vector
    # print(features,sep='\n')
    # print(type(features))
    return features


In [ ]:
filename = os.path.join(root_captioning,\
            "final8k.token.txt")
#train = loading_data(filename)
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions(os.path.join(root_caption,\
                                                          "descriptions.txt"), train_imgs)
train_features = load_features(train_imgs)
# res = list(train_descriptions.keys())[0]
# print(str(res))
# print(train_descriptions)

In [ ]:
#converting dictionary to clean list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc


In [ ]:


from keras.preprocessing.text import Tokenizer
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    # print(*desc_list,sep = "\n")
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer
# give each word an index, and store that into tokenizer.p pickle file
tokenizer = create_tokenizer(train_descriptions)
# print(tokenizer)
dump(tokenizer, open(os.path.join(root_caption,\
                                  'tokenizer2.p'), 'wb'))
tokenizer = load(open(os.path.join(root_caption,\
                                  'tokenizer.p'),"rb"))
vocab_size = len(tokenizer.word_index) + 1
vocab_size

NameError: ignored

In [ ]:
#calculate maximum length of descriptions
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)
    
max_length = max_length(descriptions)
max_length

32

In [ ]:
def data_generator(descriptions, features, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():
            #retrieve photo features
            feature = features[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, feature)
            yield [[input_image, input_sequence], output_word]


In [ ]:
def create_sequences(tokenizer, max_length, desc_list, feature):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # print(len(seq))
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            # print(len(in_seq))
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # print(in_seq)
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
#             print(vocab_size)
            # print(out_seq)
            # store
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)

In [ ]:
[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_length))
a.shape, b.shape, c.shape
# print(a)

((47, 2048), (47, 32), (47, 7576))

In [ ]:

from keras.utils.vis_utils import plot_model
# define the captioning model
def define_model(vocab_size, max_length):

    # features from the CNN model squeezed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)

    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)

    # Merging both models
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    # summarize model
    print(model.summary())

    return model

In [ ]:
# train our model
print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)

model = define_model(vocab_size, max_length)
epochs = 10
steps = len(train_descriptions)
# making a directory models to save our models
path = "/gdrive/My Drive/image caption generator/Models"
os.mkdir(path)
# os.mkdir("models")
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch= steps,     verbose=1)
    model.save(os.path.join(root_caption,
                            "models/model_" + str(i) + ".h5"))

Dataset:  7999
Descriptions: train= 7999
Photos: train= 7999
Vocabulary Size: 7576
Description Length:  32
